In [2]:
import os
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import chart_utils
import analysis_utils

In [6]:
# Download CoinMetrics data if needed

try:
    os.remove("btc.csv")
except FileNotFoundError:
    pass
!wget https://coinmetrics.io/newdata/btc.csv

--2020-11-08 22:05:38--  https://coinmetrics.io/newdata/btc.csv
Resolving coinmetrics.io (coinmetrics.io)... 104.26.14.66, 172.67.75.174, 104.26.15.66
Connecting to coinmetrics.io (coinmetrics.io)|104.26.14.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2720689 (2.6M) [application/octet-stream]
Saving to: ‘btc.csv’

btc.csv             100%[===================>]   2.59M  2.78MB/s    in 0.9s    

2020-11-08 22:05:40 (2.78 MB/s) - ‘btc.csv’ saved [2720689/2720689]



In [7]:
# data = pd.read_csv("data/05_coinbase_herfindahl_daily.csv")
data = pd.read_csv("~/Downloads/herf_block.csv")
data = analysis_utils.get_extra_datetime_cols(data, 'coinbase_block_ts', date_format="%Y-%m-%d %H:%M:%S")
price_data = pd.read_csv("btc.csv")
data = data.merge(price_data[
    ['date', 'PriceUSD']], on='date')
data.rename(columns = {'herfindal_index':'herfindahl_index'}, inplace = True)

In [8]:
daily_data = data.groupby('date', as_index=False).mean()
daily_data

,date,coinbase_block_number,coinbase_total_value,herfindahl_index,PriceUSD
0,2009-01-03,0.0,50.000000,1.000000,NaN
1,2009-01-09,7.5,50.000000,0.945197,NaN
2,2009-01-10,45.0,50.000000,1.000000,NaN
3,2009-01-11,122.0,49.999996,0.991597,NaN
4,2009-01-12,215.5,50.000000,1.000000,NaN
...,...,...,...,...,...
4318,2020-11-04,655372.0,7.533326,0.388953,14129.148195
4319,2020-11-05,655525.0,8.015552,0.361865,15606.466834
4320,2020-11-06,655675.0,7.814400,0.415411,15618.972471
4321,2020-11-07,655824.5,7.175155,0.705090,14866.356305


In [9]:
window_sizes = [1, 7, 14, 28, 56, 70]
for window_size in window_sizes:
    daily_data['herfindahl_index_{}_roll'.format(window_size)] = daily_data['herfindahl_index'].rolling(window_size).mean()
    daily_data['herfindahl_index_multiple_{}_roll'.format(window_size)] = daily_data['herfindahl_index'].rolling(window_size).mean() / daily_data['herfindahl_index'].rolling(365).mean()

In [13]:
daily_data.loc[daily_data['date'] >= '2020-01-01']

,date,coinbase_block_number,coinbase_total_value,herfindahl_index,PriceUSD,herfindahl_index_1_roll,herfindahl_index_multiple_1_roll,herfindahl_index_7_roll,herfindahl_index_multiple_7_roll,herfindahl_index_14_roll,herfindahl_index_multiple_14_roll,herfindahl_index_28_roll,herfindahl_index_multiple_28_roll,herfindahl_index_56_roll,herfindahl_index_multiple_56_roll,herfindahl_index_70_roll,herfindahl_index_multiple_70_roll
4010,2020-01-01,610777.5,12.553337,0.108646,7170.631869,0.108646,0.613103,0.108587,0.612769,0.111285,0.627997,0.117896,0.665305,0.116731,0.658725,0.127197,0.717788
4011,2020-01-02,610935.0,12.593659,0.098253,6946.825269,0.098253,0.555645,0.101093,0.571703,0.112518,0.636316,0.117456,0.664240,0.116550,0.659116,0.126476,0.715249
4012,2020-01-03,611090.0,12.592904,0.107338,7315.309360,0.107338,0.608225,0.101552,0.575440,0.113741,0.644510,0.117608,0.666424,0.116111,0.657937,0.125101,0.708881
4013,2020-01-04,611258.5,12.569025,0.085977,7343.164194,0.085977,0.488321,0.099474,0.564978,0.113300,0.643504,0.117662,0.668280,0.115313,0.654938,0.123462,0.701225
4014,2020-01-05,611424.0,12.568494,0.109071,7345.530692,0.109071,0.620404,0.102126,0.580895,0.110874,0.630657,0.116158,0.660714,0.115869,0.659066,0.122152,0.694805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,2020-11-04,655372.0,7.533326,0.388953,14129.148195,0.388953,2.472609,0.264037,1.678503,0.233721,1.485787,0.212203,1.348992,0.192522,1.223881,0.187284,1.190582
4319,2020-11-05,655525.0,8.015552,0.361865,15606.466834,0.361865,2.289714,0.293211,1.855302,0.247501,1.566070,0.218933,1.385308,0.196503,1.243380,0.189832,1.201166
4320,2020-11-06,655675.0,7.814400,0.415411,15618.972471,0.415411,2.614613,0.325107,2.046236,0.265828,1.673129,0.228901,1.440712,0.200164,1.259838,0.192800,1.213493
4321,2020-11-07,655824.5,7.175155,0.705090,14866.356305,0.705090,4.394426,0.390699,2.435007,0.302084,1.882719,0.250521,1.561355,0.209744,1.307217,0.200603,1.250245


In [18]:
chart_utils.two_axis_chart(
    daily_data[:-28], x_series='date', 
    y1_series='herfindahl_index_28_roll',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True
)

In [65]:
chart_utils.two_axis_chart(
    daily_data[:-28], x_series='date', 
    y1_series='herfindahl_index_multiple_28_roll',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
#     y2_series_axis_type='linear', y2_series_axis_range=[5000, 15000],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index Multiple',
    title='Coinbase Output Herfindahl Index Multiple',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True,
#     save_file='./docs/MinerHerfMultiple.html'
)

In [21]:
curves = pd.read_csv("data/05_miner_herf_4_curve.csv")
curves = analysis_utils.get_extra_datetime_cols(curves, 'coinbase_block_ts', date_format="%Y-%m-%d %H:%M:%S")
curves[:5]

,coinbase_tx_hash,coinbase_block_number,coinbase_block_ts,days_since_coinbase,coinbase_total_value,herfindal_index,datetime,date,year_string,month_string,week_string
0,2781047aa9c311aa8cecd141b0a3b3a316b20c0a51e6c7...,615736,2020-02-03 02:26:54,0,12.552488,0.399434,2020-02-03 02:26:54,2020-02-03,2020,February 2020,2020-02-03
1,0f9ee4085e8b90d4711eba4dfb5c32d653153dc3d01365...,626192,2020-04-16 01:33:07,0,12.830128,0.415937,2020-04-16 01:33:07,2020-04-16,2020,April 2020,2020-04-13
2,5ed5a6a4b9cae9fc79c37c47dcaa2e638a60118ab7f42f...,647384,2020-09-09 00:02:47,0,6.644689,0.132065,2020-09-09 00:02:47,2020-09-09,2020,September 2020,2020-09-07
3,75997d8275f53968e869006371cb75f53b2ad57affee46...,626977,2020-04-21 07:27:24,0,12.757197,0.502505,2020-04-21 07:27:24,2020-04-21,2020,April 2020,2020-04-20
4,fb5df9645300685a0497541da2b42436e717ccb8f91958...,646527,2020-09-03 06:15:39,0,7.161523,0.226400,2020-09-03 06:15:39,2020-09-03,2020,September 2020,2020-08-31


In [61]:
curves_day = curves.groupby(['date', 'year_string', 'month_string', 'week_string', 'days_since_coinbase'], as_index=False).herfindal_index.mean()

In [75]:
curves.loc[curves['coinbase_block_ts'] == '2020-07-23 21:02:22']

,coinbase_tx_hash,coinbase_block_number,coinbase_block_ts,days_since_coinbase,coinbase_total_value,herfindal_index,datetime,date,year_string,month_string,week_string
4042,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,1,7.195208,0.057185,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
21346,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,2,7.195183,0.057095,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
61607,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,3,7.195161,0.057094,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
118834,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,4,7.195104,0.057056,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
135326,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,5,7.195062,0.056823,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
176574,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,6,7.195059,0.056823,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
218438,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,7,7.195056,0.056823,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
267619,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,8,7.195044,0.056813,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
303682,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,9,7.195044,0.056813,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20
319679,d4b1bba809588354a6259d5471ce36a5977b655ffa3abc...,640472,2020-07-23 21:02:22,10,7.195044,0.056118,2020-07-23 21:02:22,2020-07-23,2020,July 2020,2020-07-20


In [74]:
chart_utils.miner_herf_chart(
    curves.loc[curves['coinbase_block_ts'] == '2020-07-23 21:02:22'],#.reset_index(drop=True),
#     pivot='coinbase_block_ts',
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='log',
    y1_series_axis_range=[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve: 4 deep',
    x_axis_title='Days since mined',
    data_source="Proprietary"
)

In [20]:
chart_utils.miner_herf_chart(
    curves.loc[curves['date'] >= '2020-01-01'].reset_index(drop=True),
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='linear',#'log',
    y1_series_axis_range=[0, 1],#[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve',
    x_axis_title='Days since mined',
    data_source="Proprietary"
)

In [15]:
curves_halving = pd.read_csv("data/05_miner_herf_4_curve_halvings.csv")
curves_halving[:5]

,period,days_since_coinbase,herfindal_index
0,First halving: 3 months following,0,0.956511
1,First halving: 3 months following,1,0.686177
2,First halving: 3 months following,2,0.620068
3,First halving: 3 months following,3,0.599245
4,First halving: 3 months following,4,0.585519


In [19]:
chart_utils.miner_herf_chart(
    curves_halving,
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='linear',#'log',
    y1_series_axis_range=[0, 1],#[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve',
    x_axis_title='Days since coinbase mined',
    data_source="Proprietary",
    pivot='period'
)

In [23]:
curves.sort_values(by=['coinbase_block_number', 'days_since_coinbase'], inplace=True)

In [24]:
curves

,coinbase_tx_hash,coinbase_block_number,coinbase_block_ts,days_since_coinbase,coinbase_total_value,herfindal_index,datetime,date,year_string,month_string,week_string
1939381,e12aadca7a51e816dd804536f6eb140bbf959ce7697626...,610691,2020-01-01 00:03:05,0,12.675340,1.000000,2020-01-01 00:03:05,2020-01-01,2020,January 2020,2019-12-30
12442,e12aadca7a51e816dd804536f6eb140bbf959ce7697626...,610691,2020-01-01 00:03:05,1,12.675218,0.548464,2020-01-01 00:03:05,2020-01-01,2020,January 2020,2019-12-30
40814,e12aadca7a51e816dd804536f6eb140bbf959ce7697626...,610691,2020-01-01 00:03:05,2,12.675218,0.548464,2020-01-01 00:03:05,2020-01-01,2020,January 2020,2019-12-30
76086,e12aadca7a51e816dd804536f6eb140bbf959ce7697626...,610691,2020-01-01 00:03:05,3,12.675218,0.548464,2020-01-01 00:03:05,2020-01-01,2020,January 2020,2019-12-30
115420,e12aadca7a51e816dd804536f6eb140bbf959ce7697626...,610691,2020-01-01 00:03:05,4,12.675218,0.548464,2020-01-01 00:03:05,2020-01-01,2020,January 2020,2019-12-30
...,...,...,...,...,...,...,...,...,...,...,...
1928300,f82560cc523d627f92a40d28e141ae075408271f296a04...,651138,2020-10-03 23:17:48,0,6.483738,1.000000,2020-10-03 23:17:48,2020-10-03,2020,October 2020,2020-09-28
1947296,9082de70362dee262f1231e504ca424c91b4c7fea3a6d7...,651139,2020-10-03 23:31:34,0,6.659852,1.000000,2020-10-03 23:31:34,2020-10-03,2020,October 2020,2020-09-28
1929515,6f0a5c7d46f4fa0fcf5e94f05ed2d6ae2752c837864929...,651140,2020-10-03 23:43:06,0,6.647623,1.000000,2020-10-03 23:43:06,2020-10-03,2020,October 2020,2020-09-28
1924242,cf845cde4f8befd48198ddca7418ea35e700dda7222ef0...,651141,2020-10-03 23:45:13,0,6.345682,1.000000,2020-10-03 23:45:13,2020-10-03,2020,October 2020,2020-09-28


In [67]:
cohi_day = pd.read_csv("./data/cohi_day.csv").sort_values(by='metric_date')
cohi_day['ln_change'] = np.log(cohi_day['herfindahl_index'] / cohi_day['prev_herfindahl_index'])
cohi_day = analysis_utils.get_extra_datetime_cols(cohi_day, 'metric_date', date_format="%Y-%m-%d")
cohi_day['date'] = cohi_day['metric_date']
price_data = pd.read_csv("btc.csv")
cohi_day = cohi_day.merge(price_data[
    ['date', 'PriceUSD']], on='date')
cohi_day

,metric_date,herfindahl_index,prev_herfindahl_index,mean_change,mean_pct_change,ln_change,datetime,year_string,month_string,week_string,date,PriceUSD
0,2009-01-03,1.000000,NaN,NaN,NaN,NaN,2009-01-03,2009,January 2009,2008-12-29,2009-01-03,NaN
1,2009-01-09,1.000000,1.000000,0.000000,0.000000,0.000000,2009-01-09,2009,January 2009,2009-01-05,2009-01-09,NaN
2,2009-01-10,1.000000,1.000000,0.000000,0.000000,0.000000,2009-01-10,2009,January 2009,2009-01-05,2009-01-10,NaN
3,2009-01-11,1.000000,1.000000,0.000000,0.000000,0.000000,2009-01-11,2009,January 2009,2009-01-05,2009-01-11,NaN
4,2009-01-12,0.997697,1.000000,-0.003583,-0.003583,-0.002305,2009-01-12,2009,January 2009,2009-01-12,2009-01-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4321,2020-11-07,0.253978,0.251879,-0.011128,-0.020191,0.008299,2020-11-07,2020,November 2020,2020-11-02,2020-11-07,14866.356305
4322,2020-11-08,0.252768,0.254842,-0.015023,-0.023713,-0.008173,2020-11-08,2020,November 2020,2020-11-02,2020-11-08,15500.000707
4323,2020-11-09,0.261358,0.253986,-0.009684,-0.019400,0.028613,2020-11-09,2020,November 2020,2020-11-09,2020-11-09,15319.819973
4324,2020-11-10,0.257677,0.261334,-0.017495,-0.025308,-0.014090,2020-11-10,2020,November 2020,2020-11-09,2020-11-10,15319.275676


In [76]:
cohi_day['cohi_z'] = (cohi_day['ln_change'] - cohi_day['ln_change'].rolling(365).mean()) / cohi_day['ln_change'].rolling(365).std()
cohi_day['cohi_z_28'] = cohi_day['cohi_z'].rolling(28).mean()
cohi_day['ln_change_28'] = cohi_day['ln_change'].rolling(28).mean()
cohi_day['cohi_z_7'] = cohi_day['cohi_z'].rolling(7).mean()
cohi_day['ln_change_7'] = cohi_day['ln_change'].rolling(7).mean()

In [77]:
chart_utils.two_axis_chart(
    cohi_day, x_series='date', 
    y1_series='ln_change_7',
#     y1_series_axis_type='log',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True
)

In [72]:
coinbase_big_moves = pd.read_csv("./data/coinbase_big_moves.csv").sort_values(by='metric_date')
coinbase_big_moves = analysis_utils.get_extra_datetime_cols(coinbase_big_moves, 'metric_date', date_format="%Y-%m-%d")
coinbase_big_moves['date'] = coinbase_big_moves['metric_date']
price_data = pd.read_csv("btc.csv")
coinbase_big_moves = coinbase_big_moves.merge(price_data[
    ['date', 'PriceUSD']], on='date')
coinbase_big_moves

,metric_date,num_coinbase_first_moved,num_coinbase_big_cohi_move,datetime,year_string,month_string,week_string,date,PriceUSD
0,2009-01-03,0,0,2009-01-03,2009,January 2009,2008-12-29,2009-01-03,NaN
1,2009-01-09,0,0,2009-01-09,2009,January 2009,2009-01-05,2009-01-09,NaN
2,2009-01-10,0,0,2009-01-10,2009,January 2009,2009-01-05,2009-01-10,NaN
3,2009-01-11,0,0,2009-01-11,2009,January 2009,2009-01-05,2009-01-11,NaN
4,2009-01-12,1,1,2009-01-12,2009,January 2009,2009-01-12,2009-01-12,NaN
...,...,...,...,...,...,...,...,...,...
4321,2020-11-07,99,310,2020-11-07,2020,November 2020,2020-11-02,2020-11-07,14866.356305
4322,2020-11-08,133,568,2020-11-08,2020,November 2020,2020-11-02,2020-11-08,15500.000707
4323,2020-11-09,102,269,2020-11-09,2020,November 2020,2020-11-09,2020-11-09,15319.819973
4324,2020-11-10,168,352,2020-11-10,2020,November 2020,2020-11-09,2020-11-10,15319.275676


In [78]:
chart_utils.two_axis_chart(
    coinbase_big_moves, x_series='date', 
    y1_series='num_coinbase_big_cohi_move',
#     y1_series_axis_type='log',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True
)